# **About some q-stuff**$\def\NN{\mathbb{N}}\def\KK{\mathbb{K}}\def\qbinom#1#2{\left[\begin{array}{c}#1\\#2\end{array}\right]_{q}}\def\cR{\mathcal{R}}$

Let set up some machinery to manipulate the operators and check some $q$-stuff

In [1]:
%display latex
from pseries_basis import *

The sequences that we are interested in work on the polynomials on $q$. This means we have to consider the sequences 
$$\left(\KK(q)\right)^{\NN}.$$

In this ring of sequences, the basis are sequences themselves $((a_{n,k})_k)_n$:

## **The $q$-binomial**
Can we try to have a real $q$-basis? Using the $q$-binomial, we obtain the following:

In [2]:
B = QBinomialBasis()
QPower = B.QPower(); q = B.q()

The variable `B` contains the basis of $q$-binomials, i.e., `B` is a sequence $B_k(n)$ for $k\in \NN$ where:
$$B_k(n) = \qbinom{n}{k}.$$

We have a basis that looks pretty similar to the original binomial basis we were considering before. But it is not a polynomial nor a factorial basis: we can see it is a order basis, i.e., the zeros appear at the beginning of the sequence, i.e., the zeros are at the set $\{0,\ldots,i-1\}$ for $B_i(n)$. Hence for $B_1(n)$ we would have soemhitng like $B_1(n) = a(q)n$. However, the value of $a(q)$ is not independent of $n$.

However, we have a sequence in a ring, we have a basis of the ring. We can still study the compatibility with operators.

In [3]:
B.functional_matrix(5)

[                        1                         1                         1                         1                         1]
[                        0                         1                     q + 1               q^2 + q + 1         q^3 + q^2 + q + 1]
[                        0                         0                         1               q^2 + q + 1 q^4 + q^3 + 2*q^2 + q + 1]
[                        0                         0                         0                         1         q^3 + q^2 + q + 1]
[                        0                         0                         0                         0                         1]

TODO: 1/(q;q)_n and (-q;q)_n

### **Compatibility with the shift operator $n \rightarrow n+1$**

We can find in the literature that the $q$-binomial satisfies the following recurrence equation:
$$\qbinom{n+1}{k} = q^k\qbinom{n}{k} + \qbinom{n}{k-1}.$$
We can check this identity for the first terms of our $q$-binomial basis `B` with the following code:

In [4]:
A = (QPower*B + B.shift(-1) - B.shift_in(1))
A.almost_zero(10)

True

Here we need to explain the product `QPower * B`. If we have two sequences, the product is defined as the Hadamard product. Hence, since the sequence `QPower` is the sequence $(1, q, q^2, q^3, \ldots)$; the product `QPower * B` returns $q^k \qbinom{n}{k}$. We will see later how to get with this code the sequence $\left(q^n \qbinom{n}{k}\right)_k$.

In the terms of compatibility, we have then that the $q$-binomial basis is compatible with $E_n: n \mapsto n+1$ with the compatiblity equation:
$$E_n \cdot B_k(n) = B_k(n+1) = q^k P_k(n) + P_{k-1}(n)$$
And if we write this in the recurrence form, we get:

$$\cR(E_n) = q^k + S_k$$

### **Compatibility with the multiplication by $n$**

We build (using the class `SequentialBasis`) the sequence $\left(\left(nB_k(n)\right)_n\right)_k$

In [5]:
nB = B.mult_in(LambdaSequence(lambda n : n, universe = B.base))
nB.functional_matrix(5)

[                              0                               1                               2                               3                               4]
[                              0                               1                         2*q + 2                 3*q^2 + 3*q + 3         4*q^3 + 4*q^2 + 4*q + 4]
[                              0                               0                               2                 3*q^2 + 3*q + 3 4*q^4 + 4*q^3 + 8*q^2 + 4*q + 4]
[                              0                               0                               0                               3         4*q^3 + 4*q^2 + 4*q + 4]
[                              0                               0                               0                               0                               4]

This operation is not compatible with the $q$-binomial basis since the sequence $(n)_n$ is the first element and it can not be finitely represented in terms of $q$-binomials with a fixed down coefficient.

### **Compatibility with the multiplication by $q$**

This is trivial since $q$ is part of the field we are taking the sequences from. Hence the compatibility is trivial:
$$q B_k(n)$$

### **Compatibility with the multiplication by $q^n$**

This is the key operator to consider the equalities from Ali. What would be the compatibility rule (if it exists) for the multiplication by this number. We can build the sequence $\left(\left(q^n\qbinom{n}{k}\right)_n\right)_k$ using again the class `SequenctialBasis`:

In [6]:
QB = B.mult_in(QPower)
QB.functional_matrix(5)

[                            1                             q                           q^2                           q^3                           q^4]
[                            0                             q                     q^3 + q^2               q^5 + q^4 + q^3         q^7 + q^6 + q^5 + q^4]
[                            0                             0                           q^2               q^5 + q^4 + q^3 q^8 + q^7 + 2*q^6 + q^5 + q^4]
[                            0                             0                             0                           q^3         q^7 + q^6 + q^5 + q^4]
[                            0                             0                             0                             0                           q^4]

In this case I did not find a better identity for this but the following:
$$q^n \qbinom{n}{k} = q^k\qbinom{n}{k} + q^k(q^{k+1} - 1)\qbinom{n}{k+1}.$$

This can be built again using the basis `B` and the sequence `QPower` as we did before:

In [7]:
A = QB - QPower*B - QPower*(q*QPower - 1)*B.shift(1)
A.almost_zero(10)

True

Writing this in a different fashion, we get:

$$ Q \cdot B_k(n) = (q^n)B_k(n) = q^k B_k(n) + q^k(q^{k+1}-1)B_{k+1}(n)$$

Or in a recurrence way, we can write:

$$ \cR(Q) = q^k + q^{k-1}(q^k - 1)S_k^{-1} = q^k + q^{-1}(q^{2k} - q^k)S_k^{-1} = q^k + \frac{q^k(q^k - 1)}{q}S_k^{-1}$$

## **Adapting this context to the ``ore_algebra`` package**

We have so far compatibility with 3 types of operators:
* Multiplication by the constant sequence $(q)_n$: $q: (a_n)_n \mapsto (qa_n)_n$.
* Multiplication by the power sequence $(q^n)_n$: $Q: (a_n)_n \mapsto (q^na_n)_n$.
* Shift operator w.r.t. the $n$: $E_n: (a_n)_n \mapsto (a_{n+1})_n$. We will also need the inverse of this operator $E_n^{-1}$.

We can see from these definitions that the operators $Q$ and $E_n$ do not commute:
$$E_n Q (a_n)_n = E_n (q^na_n)_n = (q^{n+1}a_{n+1})_n = q(q^n a_{n+1})_n = q Q (a_{n+1})_n = qQE_n (a_n)_n.$$

Hence we can not use the `OreAlgebra` implementation for this operators. We need something new: we have the class `OperatorAlgebra`. 

The corresponding operators can be easily obtained from the basis `B`, using diverse methods:

In [8]:
(A, (q,Q,S,Si)) = (B.OS(), (B.q(), B.Q(), B.Sn(), B.Sni()))
actions = {'Q' : lambda an : QPower * an, 'Sn' : lambda an : an.shift(1), 'Sni' : lambda an : an.shift(-1)}

At this point, we can construct the original operator $L$ as an element of the ring `A`. When we start, the operator $Q$ is the multiplication by `q^n`, while once we obtain the compatible operator, it means the multiplication by `q^k`. Something similar happens to `S`. Berfore the compatibility it is the shift w.r.t. `n` but after the compatibility transformation, it is the shift w.r.t. `k`.

* To obtain the compatible operator: ``B.recurrence(L)``
* To apply an operator to a sequence: ``L.apply(sequence, actions)``
* To obtain the solution for an operator: ``solution(L, init)`` where init is a list (with enough elements - see method `pseries_basis.misc.ore.required_init`).

In [9]:
from pseries_basis.misc.ore import solution, required_init

In [10]:
L = Q*S^2 - Q/q*S - 1

In [11]:
B.recurrence(L)

The coefficients of -1 - 1/q*Q*Sn + Q*Sn^2 are not been converted using compatibilities -- Not yet implemented


-1 + 1/q^2*Q + ((-2*q-2)/q^2)*Q^2 - 2/q*Q*Sn + ((q^2+q+1)/q^2)*Q^3 + ((q^2+q+1)/q)*Q^2*Sn + 1/q^3*Q^2*Sni + Q*Sn^2 - 2/q^3*Q^3*Sni + 1/q^3*Q^4*Sni

In [12]:
L = S - q
solution(L, [1]).almost_equals(QPower, 100)

True

## **Basic examples**

### The $q$-Pochhammer symbol (Theorem 3.3 - first identity)
    
Let us consider the $q$-polynomials that appear from the $q$-Pochhammer symbol (i.e., the correposnding to the falling factorial):

In [13]:
from pseries_basis.qbasis.qbasis import qpochhammer
C.<z> = B.base[]
BwZ = B.change_base(C)
q = BwZ.base('q'); z = BwZ.base('z')
zn = LambdaSequence(lambda n : qpochhammer(z, n, q), BwZ.base)
zn[:4]

[1,
 -z + 1,
 q*z^2 + (-q - 1)*z + 1,
 -q^3*z^3 + (q^3 + q^2 + q)*z^2 + (-q^2 - q - 1)*z + 1]

The variable `zn` contains the sequence $(z)_n = (z;q)_n$. By its simple definition, this sequence satisfies a very simple $q$-recurrence equation:
$$(z;q)_{n+1} = (1-zq^{n})(z;q)_n,$$
which, in terms of our operators mean:
$$L \cdot (z;q)_n = (E_n - 1 + zQ)\cdot (z;q)_n = 0$$

In [14]:
Az = A.change_ring(C)
Q,S,Si = Az.gens()
L = S - 1 + z*Q
solution(L, zn[:required_init(L)]).almost_equals(zn, 50)

True

Since the operators $Q$, $S$ and $Si$ are compatible with the $q$-binomial basis, we can compute a $q$-recurrence equation when we write:
$$(z;q)_n = \sum_{k=0}^n c_k \qbinom{n}{k},$$
for the sequence $(c_k)_k$. This recurrence equation is obtained by substituing each operator in $L$ with its compatibility condition:

In [15]:
L_comp = BwZ.recurrence(L)
show(L_comp)
cn = solution(L_comp, BwZ.functional_to_self(zn, required_init(L_comp)))
cn[:5]

The coefficients of -1 + z*Q + Sn are not been converted using compatibilities -- Not yet implemented


-1 + (z+1)*Q + Sn - 1/q*z*Q*Sni + 1/q*z*Q^2*Sni

[1, -z, q*z^2, -q^3*z^3, q^6*z^4]

We can check that this recurrence is satisfied by the sequence 
$$c_n = (-1)^n z^n q^{(n^2-n)/2}$$

(Note: we would need to implement closure properties to these operators in order to check symbolically and not only check the first terms)

In [16]:
cn_real = LambdaSequence(lambda n : (-1)**n * z**n * q**((n**2-n)/2), BwZ.base)
cn.almost_equals(cn, 50)

True

### Same sequence, different basis (Theorem 3.3 - second identity)

The second part of Theorem 3.3 is the following identity:
$$(z;q)_n = \sum_{j=0}^n z^j \qbinom{n+j+1}{j}.$$

Using the class `QBinomialBasis` we can still prove this identity:

In [25]:
B = QBinomialBasis(C, a = 1, b = 1, c = -1); show(B)
B.qrecurrences

ValueError: The value for the parameter 'c' must be an non-negative integer.

In [24]:
B.shift_in(1) - B.QPower()*B + B.shift(1)

Sequence over [Set of Sequences from NN to Univariate Polynomial Ring in z over Fraction Field of Univariate Polynomial Ring in q over Rational Field]: (Sequence over [Univariate Polynomial Ring in z over Fraction Field of Univariate Polynomial Ring in q over Rational Field]: (q + 1, q^2 + q + 1, q^3 + q^2 + q + 1,...), Sequence over [Univariate Polynomial Ring in z over Fraction Field of Univariate Polynomial Ring in q over Rational Field]: (q^2 + q + 2, q^4 + q^3 + 2*q^2 + q + 2, q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 2,...), Sequence over [Univariate Polynomial Ring in z over Fraction Field of Univariate Polynomial Ring in q over Rational Field]: (q^3 + 2*q^2 + 2*q + 2, q^6 + q^5 + 2*q^4 + 3*q^3 + 3*q^2 + 2*q + 2, q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 4*q^4 + 4*q^3 + 3*q^2 + 2*q + 2,...),...)

In [27]:
1/QPower

TypeError: unsupported operand parent(s) for /: 'Set of Sequences from NN to Fraction Field of Univariate Polynomial Ring in q over Rational Field' and 'Set of Sequences from NN to Fraction Field of Univariate Polynomial Ring in q over Rational Field'

### A more ellaborate identity (Theorem 3.4 - identity 3.3.8)

Consider now the following sequence:
$$a_m = \left\{\begin{array}{ll}
    (q;q^2)_n & \text{if }m = 2n\\
    0 & \text{otherwise}
\end{array}\right.$$

In [20]:
q = B.base('q')
def am_func(m):
    if m % 2 == 0:
        return qpochhammer(q,m//2, q^2)
    return 0
am = LambdaSequence(am_func, B.base)
am[:5]

[1, 0, -q + 1, 0, q^4 - q^3 - q + 1]

Now, we want to write this sequence using the $q$-binomial basis, in the following way:
$$a_m = \sum_{k = 0}^m c_k \qbinom{m}{k}.$$

If we manage to obtain an operator $L$ that annihilates $a_m$, then we can compute an operator that annihilates $(c_k)_k$.

This operator $L$ can be computed easily from the definition of $a_m$, obtaining:
$$L = E_m^2 - 1 + qQ.$$
We check this using the following code:

In [21]:
Q, S, Si = A.gens()
L = S^2 - 1 + q*Q
L.apply(am, actions).almost_zero(50)

True

Hence we can compute the recursion for $(c_k)_k$ by substituing the compatibility operators into $L$:

In [22]:
L_comp = B.recurrence(L)
show(L_comp)
ck = solution(L_comp, B.functional_to_self(am, required_init(L_comp)))
ck[:10]

The coefficients of -1 + q*Q + Sn^2 are not been converted using compatibilities -- Not yet implemented


-1 + q*Q + Q^2 + (q+1)*Q*Sn - Q*Sni + Sn^2 + Q^2*Sni

[1, -1, 1, -1, 1, -1, 1, -1, 1, -1]

In the theorem 3.4, indentity 3.3.8, they claim the sequence $c_k = (-1)^k$. Using closure properties over $L_{comp}$ and the annihilator of $(-1)^k$ (which would be $S + 1$) we could prove symbolically that these two sequence are the same. However, we will simply check that $(-1)^k$ isannihilated by our $L_{comp}$: 

In [23]:
ck_real = LambdaSequence(lambda k : (-1)**k, B.base)
ck.almost_equals(ck_real, 100)

True

### A weird binomial identity (Theorem 3.4 - identity 3.3.9)

In this example, we have the following identity:
$$\qbinom{n+m+1}{m+1} = \sum_{j=0}^n q^j\qbinom{m+j}{m}.$$

This identity looks like something we can prove using our code. However, we have a couple of issues with this:

* The $q$-binomial used in this identity do not have the summing parameter in the bottom: we need to adapt the compatibilities for this type of basis.
* $n$ is a parameter. Hence we are going to add two parameters to the base field: $n$ and $q^n$.

Let us start by obtaining a valid field for this identity and then, we will compute a recurrence that annihilates the left-hand side of the identity:

In [37]:
from pseries_basis.qbasis.qbasis import qbinomial
D = QQ['n','q_n'].fraction_field(); n,qn = D.gens()
aux_B = QBasis(D, LambdaSequence(lambda *n : 1, universe = QQ, dim=2)); q = aux_B.q(); F = aux_B.base
lhs = LambdaSequence(lambda m : prod([(1-qn*q**(m+1-i))/(1-q**(i+1)) for i in range(m+1)]), universe = F)

In [48]:
Q, Sn, Sni = aux_B.Q(), aux_B.Sn(), aux_B.Sni()
L_lhs = (1-q**2*Q)*Sn - (q**2*(1-qn)*Q+1-q**2*Q); show(L_lhs)
sol_lhs = solution(L_lhs, [lhs[0]])
sol_lhs.almost_equals(lhs, 20)

-1 + q_n*q^2*Q + Sn + (-q^2)*Q*Sn

True

The operator we got is pretty simple, although it involves the constant $q^n$. Now we need to define the basis of the right-hand side of the identity. This basis should involve everything that involves the variabl $m$. In this case, it is simply the $q$-binomial coefficient $\qbinom{m+j}{m}$. Unfortunately, this is not yet included in the class `QBinomialBasis`. Let us build this basis from scratch:

In [51]:
B = QBasis(F, LambdaSequence(lambda j, m : qbinomial(m+j,m,q), F, 2), False)
B.functional_matrix(5)

[                                                                                                                               1                                                                                                                                1                                                                                                                                1                                                                                                                                1                                                                                                                                1]
[                                                                                                                               1                                                                                                                            q + 1                                                                                                                      q^2 + q + 1                                                                                                                q^3 + q^2 + q + 1                                                                                                          q^4 + q^3 + q^2 + q + 1]
[                                                                                                                               1                                                                                                                      q^2 + q + 1                                                                                                        q^4 + q^3 + 2*q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                                q^3 + q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                          q^4 + q^3 + q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1 q^16 + q^15 + 2*q^14 + 3*q^13 + 5*q^12 + 5*q^11 + 7*q^10 + 7*q^9 + 8*q^8 + 7*q^7 + 7*q^6 + 5*q^5 + 5*q^4 + 3*q^3 + 2*q^2 + q + 1]

Now we need to discover the compatibilities with the usual operators: $S_n$ and multiplication by the sequence $q^m$:

In [54]:
B.mult_in(B.QPower()).functional_matrix(5)

[                                                                                                                                       1                                                                                                                                        q                                                                                                                                      q^2                                                                                                                                      q^3                                                                                                                                      q^4]
[                                                                                                                                       1                                                                                                                                  q^2 + q                                                                                                                          q^4 + q^3 + q^2                                                                                                                    q^6 + q^5 + q^4 + q^3                                                                                                              q^8 + q^7 + q^6 + q^5 + q^4]
[                                                                                                                                       1                                                                                                                            q^3 + q^2 + q                                                                                                            q^6 + q^5 + 2*q^4 + q^3 + q^2                                                                                            q^9 + q^8 + 2*q^7 + 2*q^6 + 2*q^5 + q^4 + q^3                                                                         q^12 + q^11 + 2*q^10 + 2*q^9 + 3*q^8 + 2*q^7 + 2*q^6 + q^5 + q^4]
[                                                                                                                                       1                                                                                                                      q^4 + q^3 + q^2 + q                                                                                            q^8 + q^7 + 2*q^6 + 2*q^5 + 2*q^4 + q^3 + q^2                                                                 q^12 + q^11 + 2*q^10 + 3*q^9 + 3*q^8 + 3*q^7 + 3*q^6 + 2*q^5 + q^4 + q^3                                     q^16 + q^15 + 2*q^14 + 3*q^13 + 4*q^12 + 4*q^11 + 5*q^10 + 4*q^9 + 4*q^8 + 3*q^7 + 2*q^6 + q^5 + q^4]
[                                                                                                                                       1                                                                                                                q^5 + q^4 + q^3 + q^2 + q                                                                           q^10 + q^9 + 2*q^8 + 2*q^7 + 3*q^6 + 2*q^5 + 2*q^4 + q^3 + q^2                                      q^15 + q^14 + 2*q^13 + 3*q^12 + 4*q^11 + 4*q^10 + 5*q^9 + 4*q^8 + 4*q^7 + 3*q^6 + 2*q^5 + q^4 + q^3 q^20 + q^19 + 2*q^18 + 3*q^17 + 5*q^16 + 5*q^15 + 7*q^14 + 7*q^13 + 8*q^12 + 7*q^11 + 7*q^10 + 5*q^9 + 5*q^8 + 3*q^7 + 2*q^6 + q^5 + q^4]

In [81]:
Matrix([[(B)[i][j] for j in range(5)] for i in range(5)])

[                                                                                                                               1                                                                                                                                1                                                                                                                                1                                                                                                                                1                                                                                                                                1]
[                                                                                                                               1                                                                                                                            q + 1                                                                                                                      q^2 + q + 1                                                                                                                q^3 + q^2 + q + 1                                                                                                          q^4 + q^3 + q^2 + q + 1]
[                                                                                                                               1                                                                                                                      q^2 + q + 1                                                                                                        q^4 + q^3 + 2*q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                                q^3 + q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                          q^4 + q^3 + q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1 q^16 + q^15 + 2*q^14 + 3*q^13 + 5*q^12 + 5*q^11 + 7*q^10 + 7*q^9 + 8*q^8 + 7*q^7 + 7*q^6 + 5*q^5 + 5*q^4 + 3*q^3 + 2*q^2 + q + 1]

In [72]:
Matrix([[(B.shift_in(1))[i][j] for j in range(5)] for i in range(5)])

[                                                                                                                                                                        1                                                                                                                                                                         1                                                                                                                                                                         1                                                                                                                                                                         1                                                                                                                                                                         1]
[                                                                                                                                                                    q + 1                                                                                                                                                               q^2 + q + 1                                                                                                                                                         q^3 + q^2 + q + 1                                                                                                                                                   q^4 + q^3 + q^2 + q + 1                                                                                                                                             q^5 + q^4 + q^3 + q^2 + q + 1]
[                                                                                                                                                              q^2 + q + 1                                                                                                                                                 q^4 + q^3 + 2*q^2 + q + 1                                                                                                                                 q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                                                                 q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                                                q^10 + q^9 + 2*q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                                                                                                                                        q^3 + q^2 + q + 1                                                                                                                                 q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                                                         q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q + 1                                                                              q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1                                                   q^15 + q^14 + 2*q^13 + 3*q^12 + 4*q^11 + 5*q^10 + 6*q^9 + 6*q^8 + 6*q^7 + 6*q^6 + 5*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1]
[                                                                                                                                                  q^4 + q^3 + q^2 + q + 1                                                                                                                 q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                              q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1                                          q^16 + q^15 + 2*q^14 + 3*q^13 + 5*q^12 +

In [77]:
Matrix([[(B)[i][j] for j in range(5)] for i in range(5)])

[                                                                                                                               1                                                                                                                                1                                                                                                                                1                                                                                                                                1                                                                                                                                1]
[                                                                                                                               1                                                                                                                            q + 1                                                                                                                      q^2 + q + 1                                                                                                                q^3 + q^2 + q + 1                                                                                                          q^4 + q^3 + q^2 + q + 1]
[                                                                                                                               1                                                                                                                      q^2 + q + 1                                                                                                        q^4 + q^3 + 2*q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                                q^3 + q^2 + q + 1                                                                                        q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1                                                                q^9 + q^8 + 2*q^7 + 3*q^6 + 3*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1]
[                                                                                                                               1                                                                                                          q^4 + q^3 + q^2 + q + 1                                                                        q^8 + q^7 + 2*q^6 + 2*q^5 + 3*q^4 + 2*q^3 + 2*q^2 + q + 1                                     q^12 + q^11 + 2*q^10 + 3*q^9 + 4*q^8 + 4*q^7 + 5*q^6 + 4*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q + 1 q^16 + q^15 + 2*q^14 + 3*q^13 + 5*q^12 + 5*q^11 + 7*q^10 + 7*q^9 + 8*q^8 + 7*q^7 + 7*q^6 + 5*q^5 + 5*q^4 + 3*q^3 + 2*q^2 + q + 1]

## **Examples from Ali**

### Smaller example

Let us consider the solution of the $q$-recurrence equation:
$$qy_n = q^{2n+1}(1+q) y_{n-1} - q^{3n+1}(q^{n-1} - 1)y_{n-2}$$
We create now the corresponding operator and its solution $(y_n)_n$:

In [61]:
L = Q^3*(-q+Q)*Si^2 - q*Q^2*(1+q)*Si + q
L = (S^2*L).canonical()
yn = solution(L, [1, q^2*(1+q)])

In [62]:
yn[:5]

[1,
 q^3 + q^2,
 q^8 + q^7 + 2*q^6,
 q^15 + q^14 + 2*q^13 + 3*q^12 + q^11,
 q^24 + q^23 + 2*q^22 + 3*q^21 + 5*q^20 + 2*q^19 + 2*q^18]

If we want to write now the solution:
$$y_n = \sum_{k=0}^n c_k \qbinom{n}{k},$$
then we can use the method `B.recurrence` to obtain a recurrence for $c_k$ and also obtain the first terms by using the method `B.funcitonal_to_self`.

In the following cell we compute the recurrence for $c_k$ and we display the degrees of this new recurrence with respect to $Q \equiv q^n$, $Sn$ and $Sn^{-1}$:

In [63]:
L2 = B.recurrence(L); 
show(L2)
L2.degrees()

The coefficients of q*Sn^2 - q^7*Q^3 + (-q^6-q^5)*Q^2*Sn + q^8*Q^4 are not been converted using compatibilities -- Not yet implemented


(q^5+2*q^4+q^3+q)*Q^2 + (q^2+q)*Q*Sn + q*Sn^2 + (-q^7-q^6-2*q^5-2*q^4-q^3)*Q^3 + (-q^6-q^5)*Q^2*Sn + (-q^3-q^2)*Q^2*Sni + q^8*Q^4 + (q^6+q^5+2*q^4+3*q^3+3*q^2+q)*Q^3*Sni + (-q^7-2*q^6-2*q^5-3*q^4-2*q^3-2*q^2-q)*Q^4*Sni + (-q^4-q^3-q^2-q-1)*Q^3*Sni^2 + (q^7+q^6+q^5+q^4)*Q^5*Sni + ((q^6+2*q^5+4*q^4+3*q^3+3*q^2+2*q+1)/q)*Q^4*Sni^2 + q*Q^3*Sni^3 + ((-q^6-2*q^5-4*q^4-4*q^3-3*q^2-2*q-1)/q)*Q^5*Sni^2 + ((-q^3-2*q^2-2*q-2)/q)*Q^4*Sni^3 + (q^4+q^3+2*q^2+q+1)*Q^6*Sni^2 + ((q^5+2*q^4+4*q^3+4*q^2+3*q+1)/q^3)*Q^5*Sni^3 + 1/q^2*Q^4*Sni^4 + ((-q^5-2*q^4-3*q^3-4*q^2-2*q-1)/q^4)*Q^6*Sni^3 + ((-q^3-q^2-q-1)/q^5)*Q^5*Sni^4 + ((q^3+q^2+q+1)/q^4)*Q^7*Sni^3 + ((q^4+q^3+2*q^2+q+1)/q^7)*Q^6*Sni^4 + ((-q^3-q^2-q-1)/q^8)*Q^7*Sni^4 + 1/q^8*Q^8*Sni^4

(8, 2, 4)

We can then compute the solution $c_k$ using again the method `solution`:

In [64]:
ck = solution(L2, B.functional_to_self(sol, required_init(L2)))
for i in range(6):
    print(f"{i} -- {ck[i]}")

0 -- 1
1 -- q^3 + q^2 - 1
2 -- q^8 + q^7 + 2*q^6 - q^4 - 2*q^3 - q^2 + q
3 -- q^15 + q^14 + 2*q^13 + 3*q^12 + q^11 - q^10 - 2*q^9 - 4*q^8 - 3*q^7 - q^6 + 2*q^5 + 2*q^4
4 -- q^24 + q^23 + 2*q^22 + 3*q^21 + 5*q^20 + 2*q^19 + q^18 - 2*q^17 - 4*q^16 - 7*q^15 - 7*q^14 - 5*q^13 - 2*q^12 + 4*q^11 + 5*q^10 + 5*q^9 + q^8 - q^6 - q^5
5 -- q^35 + q^34 + 2*q^33 + 3*q^32 + 5*q^31 + 7*q^30 + 5*q^29 + 3*q^28 + q^27 - 3*q^26 - 7*q^25 - 12*q^24 - 13*q^23 - 13*q^22 - 10*q^21 - 4*q^20 + 5*q^19 + 10*q^18 + 13*q^17 + 12*q^16 + 6*q^15 + q^14 - 3*q^13 - 4*q^12 - 4*q^11 - 2*q^10 + q^8


### Old example

The example by Ali is a recurrence with plenty of components:

$$\begin{array}{rll}
q^{19 + 6n}\left({\left(q^{24} + q^{23} + q^{22}\right)} q^{6 \, n} - {\left(q^{20} + q^{19} + q^{18}\right)} q^{5 \, n} + {\left(q^{17} + q^{16} + q^{15}\right)} q^{4 \, n} + {\left(q^{12} + q^{11} + q^{10}\right)} q^{3 \, n} - {\left(q^{9} + q^{8} + q^{7}\right)} q^{2 \, n} + {\left(q^{5} + 2 \, q^{4} + q^{3}\right)} q^{n} - 1\right) & f(n) + &  \\
q^{5 \, n + 19}\left({\left(q^{28} + q^{27} + q^{26} + q^{25} + q^{24} + q^{23}\right)} q^{8 \, n} - {\left(q^{21} + q^{20} + q^{19}\right)} q^{7 \, n} + {\left(q^{21} + q^{20} + q^{19} + q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} + {\left(q^{16} + q^{15} + q^{14} + q^{13} + q^{12} + q^{11}\right)} q^{5 \, n} + {\left(q^{15} + q^{14} + q^{13} - q^{10} - q^{9} - q^{8}\right)} q^{4 \, n} - {\left(q^{12} + q^{11} - q^{9} - 2 \, q^{8} - 2 \, q^{7} - 2 \, q^{6} - 2 \, q^{5} - q^{4}\right)} q^{3 \, n} + {\left(q^{8} - q^{5} - q\right)} q^{2 \, n} + {\left(q^{3} + q^{2} + q\right)} q^{n} - 1\right) & f(1 + n) + & \\
q^{3 \, n + 12} \left( {\left(q^{37} + q^{36} + q^{35}\right)} q^{11 \, n} - {\left(q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{32} + 2 \, q^{31} + 3 \, q^{30} + 3 \, q^{29} + 2 \, q^{28} + q^{27}\right)} q^{9 \, n} - {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + 2 \, q^{26} - q^{23}\right)} q^{8 \, n} + {\left(q^{27} + 2 \, q^{26} + 3 \, q^{25} + 3 \, q^{24} + 2 \, q^{23} + q^{22} - q^{21} - q^{20} - q^{19}\right)} q^{7 \, n} - {\left(q^{25} + 2 \, q^{24} + 4 \, q^{23} + 2 \, q^{22} + 2 \, q^{21} - q^{20} - 2 \, q^{19} - 4 \, q^{18} - 4 \, q^{17} - 3 \, q^{16} - q^{15}\right)} q^{6 \, n} + {\left(q^{21} + 2 \, q^{20} + 2 \, q^{19} - q^{17} - 3 \, q^{16} - 2 \, q^{15} - 3 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} - {\left(q^{18} + q^{17} + 2 \, q^{16} - q^{14} - 3 \, q^{13} - 3 \, q^{12} - 3 \, q^{11} - 2 \, q^{10} - q^{9}\right)} q^{4 \, n} - {\left(q^{13} + q^{12} + 3 \, q^{11} + 3 \, q^{10} + 3 \, q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} + {\left(q^{10} + q^{9} + 2 \, q^{8} + 2 \, q^{7} + q^{6}\right)} q^{2 \, n} - {\left(q^{6} + 2 \, q^{5} + 3 \, q^{4} + 2 \, q^{3}\right)} q^{n} + q + 1 \right) & f(2 + n) + & \\
q^{2 \, n + 10} \left( {\left(q^{40} + q^{39} + q^{38}\right)} q^{12 \, n} - {\left(q^{37} + 2 \, q^{36} + 2 \, q^{35} + q^{34}\right)} q^{11 \, n} + {\left(q^{35} + q^{34} + 2 \, q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + q^{26}\right)} q^{9 \, n} - {\left(q^{29} + q^{28} + 3 \, q^{27} + 4 \, q^{26} + 6 \, q^{25} + 5 \, q^{24} + 3 \, q^{23} + q^{22}\right)} q^{8 \, n} + {\left(q^{26} + 4 \, q^{25} + 5 \, q^{24} + 7 \, q^{23} + 7 \, q^{22} + 7 \, q^{21} + 5 \, q^{20} + 3 \, q^{19} + q^{18}\right)} q^{7 \, n} - {\left(2 \, q^{22} + 3 \, q^{21} + 3 \, q^{20} + 3 \, q^{19} + 2 \, q^{18} + 2 \, q^{17} + q^{16} + q^{15}\right)} q^{6 \, n} + {\left(q^{18} - q^{15} - 2 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} + {\left(2 \, q^{15} + 4 \, q^{14} + 7 \, q^{13} + 7 \, q^{12} + 8 \, q^{11} + 6 \, q^{10} + 4 \, q^{9} + q^{8}\right)} q^{4 \, n} - {\left(q^{12} + 3 \, q^{11} + 5 \, q^{10} + 6 \, q^{9} + 5 \, q^{8} + 4 \, q^{7} + 2 \, q^{6} + q^{5}\right)} q^{3 \, n} + {\left(q^{8} + 2 \, q^{7} + 5 \, q^{6} + 5 \, q^{5} + 4 \, q^{4} + 2 \, q^{3} + q^{2}\right)} q^{2 \, n} - q^{n + 2} - 1 \right) & f(3 + n) + & \\
q^{n + 4} - 1 \left( {\left(q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} - {\left(q^{15} + q^{14} + q^{13}\right)} q^{5 \, n} + {\left(q^{13} + q^{12} + q^{11}\right)} q^{4 \, n} + {\left(q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} - {\left(q^{7} + q^{6} + q^{5}\right)} q^{2 \, n} + {\left(q^{4} + 2 \, q^{3} + q^{2}\right)} q^{n} - 1 \right) & f[4 + n] & = 0
\end{array}$$

In [16]:
L = (-Q^3*(q+Q)*(-q+Q)*(-q^3+Q^2)*(1-Q+Q^2)*Si^2) + (-q^2*Q^2*(q^2 + Q^3 - Q^5 + Q^6 + q^2*Q + q^3*Q - q*Q^2 - q^2*Q^2 + q*Q^3 + q*Q^4 +  q^2*Q^4 - q*Q^5)) + (q^2*(q^2 + Q^2 - q*Q))
L = (S^2*L).canonical()

In [17]:
L(**compatibilities).canonical()

(-q^5+2*q^4+2*q^3)*Q^2 + (q^5+2*q^4)*Q*S + q^4*S^2 + (-q^11-q^8-q^7-q^5-2*q^4-3*q^3-q^2)*Q^3 + (q^7-2*q^5-2*q^4)*Q^2*S - q^5*Q*S^2 + (q^13+q^10+4*q^9+6*q^8+8*q^7+7*q^6+5*q^5+3*q^4+q^3+q^2)*Q^4 + (q^10+q^9+q^8+q^7+q^5+q^4)*Q^3*S + (q^10+q^9+q^8+q^5-q^3+2*q^2+q)*Q^3*Si + (-q^8+q^6)*Q^2*S^2 + (-q^15+q^13+q^12-2*q^11-6*q^10-10*q^9-14*q^8-16*q^7-14*q^6-10*q^5-6*q^4-2*q^3)*Q^5 + (-q^12-3*q^11-4*q^10-4*q^9-4*q^8-2*q^7)*Q^4*S + (-q^12-q^11-2*q^10-2*q^9-q^8-q^7-4*q^6-6*q^5-5*q^4-4*q^3-4*q^2-2*q-1)*Q^4*Si + (-q^11-q^10)*Q^3*S^2 + (-q^8-q^7-q^6)*Q^3*Si^2 + (-q^15-q^14-q^13+2*q^11+4*q^10+6*q^9+9*q^8+11*q^7+10*q^6+8*q^5+6*q^4+4*q^3+q^2)*Q^6 + (q^13+3*q^12+4*q^11+4*q^10+4*q^9+4*q^8+2*q^7)*Q^5*S + (q^14+q^13+q^12-q^9+6*q^7+14*q^6+19*q^5+21*q^4+19*q^3+12*q^2+7*q+3)*Q^5*Si + (q^12+q^11)*Q^4*S^2 + ((q^11+q^10+3*q^9+3*q^8+3*q^7+q^6+q^4+3*q^3+q^2+1)/q)*Q^4*Si^2 + q^5*Q^3*Si^3 + (q^17+q^16+q^15+3*q^14+6*q^13+8*q^12+10*q^11+12*q^10+13*q^9+11*q^8+8*q^7+6*q^6+4*q^5+2*q^4-q^2)*Q^7 + (q^15+q^14-q^9-2*q^8-q^7)*Q^6*S + ((q^15+2*q^14+3*q^13+4*q^12+4*q^11-q^10-8*q^9-17*q^8-25*q^7-28*q^6-27*q^5-23*q^4-15*q^3-8*q^2-4*q-1)/q^2)*Q^6*Si + (-q^13-q^12)*Q^5*S^2 + ((-q^14-q^13-2*q^12-2*q^11-2*q^10-q^9+q^7-6*q^5-12*q^4-12*q^3-9*q^2-7*q-3)/q^2)*Q^5*Si^2 + (-q^7-q^6-2*q^5-2*q^4-q^3)*Q^4*Si^3 + (-q^16-6*q^15-9*q^14-13*q^13-16*q^12-20*q^11-23*q^10-25*q^9-22*q^8-19*q^7-15*q^6-12*q^5-8*q^4-5*q^3-q^2)*Q^8 + (-q^17-3*q^16-4*q^15-4*q^14-4*q^13-4*q^12-4*q^11-4*q^10-2*q^9)*Q^7*S + ((-q^19-2*q^18-3*q^17-3*q^16-4*q^15-7*q^14-10*q^13-11*q^12-12*q^11-13*q^10-11*q^9-8*q^8-5*q^7-q^6+2*q^5+3*q^4+3*q^3+2*q^2+q+1)/q^3)*Q^7*Si + (-q^16-q^15)*Q^6*S^2 + ((-q^15-2*q^14-5*q^13-7*q^12-10*q^11-8*q^10-3*q^9+8*q^8+20*q^7+28*q^6+28*q^5+24*q^4+17*q^3+8*q^2+4*q+1)/q^5)*Q^6*Si^2 + ((q^12+q^11+2*q^10+2*q^9+2*q^8+q^7-q^5-2*q^4-q^3+2*q^2+4*q+2)/q^3)*Q^5*Si^3 + q^3*Q^4*Si^4 + (-q^18+2*q^17+4*q^16+6*q^15+8*q^14+10*q^13+12*q^12+14*q^11+16*q^10+16*q^9+14*q^8+12*q^7+10*q^6+8*q^5+6*q^4+4*q^3+2*q^2)*Q^9 + (q^18+3*q^17+3*q^16+3*q^15+3*q^14+3*q^13+3*q^12+3*q^11+3*q^10+q^9)*Q^8*S + ((q^20+q^19+q^18+2*q^17+7*q^16+13*q^15+21*q^14+30*q^13+41*q^12+52*q^11+59*q^10+61*q^9+59*q^8+54*q^7+45*q^6+33*q^5+23*q^4+15*q^3+8*q^2+4*q+1)/q^4)*Q^8*Si + (q^17+q^16)*Q^7*S^2 + ((q^21+2*q^20+4*q^19+5*q^18+8*q^17+10*q^16+14*q^15+17*q^14+20*q^13+20*q^12+20*q^11+13*q^10+3*q^9-5*q^8-11*q^7-15*q^6-13*q^5-10*q^4-7*q^3-3*q^2-q-1)/q^7)*Q^7*Si^2 + ((2*q^13+4*q^12+7*q^11+10*q^10+11*q^9+11*q^8+8*q^7-q^6-9*q^5-12*q^4-10*q^3-7*q^2-4*q-1)/q^7)*Q^6*Si^3 + ((-q^7-q^6-q^5+q^2+2*q+1)/q^2)*Q^5*Si^4 + (-q^18-q^17-2*q^16-2*q^15-3*q^14-3*q^13-4*q^12-4*q^11-5*q^10-4*q^9-4*q^8-3*q^7-3*q^6-2*q^5-2*q^4-q^3-q^2)*Q^10 + (-q^19-q^18-q^17-q^16-q^15-q^14-q^13-q^12-q^11-q^10)*Q^9*S + ((q^22+q^21+2*q^20-4*q^18-10*q^17-17*q^16-27*q^15-38*q^14-52*q^13-64*q^12-71*q^11-74*q^10-74*q^9-70*q^8-62*q^7-51*q^6-38*q^5-27*q^4-18*q^3-11*q^2-5*q-2)/q^5)*Q^9*Si - q^18*Q^8*S^2 + ((-q^23-2*q^22-4*q^21-5*q^20-7*q^19-10*q^18-16*q^17-21*q^16-30*q^15-42*q^14-57*q^13-67*q^12-76*q^11-79*q^10-79*q^9-70*q^8-60*q^7-46*q^6-34*q^5-22*q^4-14*q^3-7*q^2-4*q-1)/q^9)*Q^8*Si^2 + ((-q^21-2*q^20-4*q^19-6*q^18-10*q^17-13*q^16-18*q^15-21*q^14-26*q^13-29*q^12-30*q^11-26*q^10-19*q^9-8*q^8+3*q^7+10*q^6+11*q^5+9*q^4+6*q^3+3*q^2+q+1)/q^10)*Q^7*Si^3 + ((-q^11-2*q^10-5*q^9-7*q^8-9*q^7-8*q^6-7*q^5-4*q^4-2*q^3+q^2+3*q+1)/q^8)*Q^6*Si^4 + ((q^3-q-1)/q^3)*Q^5*Si^5 + ((-q^23-q^22+2*q^20+5*q^19+8*q^18+13*q^17+18*q^16+24*q^15+32*q^14+37*q^13+40*q^12+42*q^11+42*q^10+40*q^9+37*q^8+32*q^7+25*q^6+19*q^5+14*q^4+9*q^3+6*q^2+3*q+1)/q^6)*Q^10*Si + ((-q^26-q^25-2*q^24-2*q^23-3*q^22-2*q^21+q^20+8*q^19+19*q^18+36*q^17+59*q^16+87*q^15+114*q^14+138*q^13+156*q^12+167*q^11+166*q^10+153*q^9+133*q^8+108*q^7+83*q^6+58*q^5+37*q^4+22*q^3+12*q^2+5*q+2)/q^11)*Q^9*Si^2 + ((q^24+2*q^23+5*q^22+8*q^21+13*q^20+17*q^19+22*q^18+26*q^17+33*q^16+38*q^15+47*q^14+55*q^13+62*q^12+66*q^11+68*q^10+62*q^9+55*q^8+49*q^7+39*q^6+29*q^5+21*q^4+13*q^3+7*q^2+4*q+1)/q^13)*Q^8*Si^3 + ((q^19+2*q^18+4*q^17+6*q^16+10*q^15+13*q^14+18*q^13+21*q^12+25*q^11+25*q^10+24

In [19]:
Binomial

In [26]:
L3 = ((S^9)*L2).canonical()

In [31]:
for k,v in L3.monomial_coefficients().items():
    print(f"{k} -- {v}")

Q^16*S^3 -- -q^114 - q^113 - 2*q^112 - 2*q^111 - 3*q^110 - 3*q^109 - 4*q^108 - 4*q^107 - 5*q^106 - 4*q^105 - 4*q^104 - 3*q^103 - 3*q^102 - 2*q^101 - 2*q^100 - q^99 - q^98
Q^17*S^2 -- -q^108 - q^107 - q^106 - q^105 - q^104 - q^103 - q^102 - q^101 - q^100 - q^99
Q^18*S -- -q^100
Q^15*S^3 -- -q^117 - q^116 - 2*q^115 - 3*q^114 - 4*q^113 - 5*q^112 - 7*q^111 - 6*q^110 - 6*q^109 - 4*q^108 - 2*q^107 + 2*q^106 + 5*q^105 + 12*q^104 + 18*q^103 + 22*q^102 + 25*q^101 + 27*q^100 + 27*q^99 + 27*q^98 + 25*q^97 + 22*q^96 + 17*q^95 + 12*q^94 + 9*q^93 + 6*q^92 + 4*q^91 + 2*q^90 + q^89
Q^16*S^2 -- -q^113 - q^112 - 2*q^111 - 2*q^110 - 3*q^109 - 3*q^108 - 4*q^107 - 4*q^106 - 3*q^105 - q^104 + 2*q^102 + 3*q^101 + 5*q^100 + 6*q^99 + 9*q^98 + 9*q^97 + 7*q^96 + 6*q^95 + 5*q^94 + 4*q^93 + 3*q^92 + 2*q^91 + q^90
Q^17*S -- -q^107 - q^106 - q^105 - q^104 - q^103 - q^102 - q^101 - q^100 - q^99 + q^98 + q^97 + q^96 + q^95 + q^94 + q^93 + q^92 + q^91
Q^15*S^4 -- -q^118 - q^117 - 2*q^116 - 3*q^115 - 4*q^114 - 5*q^113 -

In [24]:
q,n = var('q,n')
coeffs = [
    (-q^(19 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + q^(24 + 7*n) - q^(26 + 8*n) - q^(27 + 8*n) - q^(28 + 8*n) + q^(29 + 9*n) + # f(n)
     q^(30 + 9*n) + q^(31 + 9*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(36 + 10*n) - q^(37 + 11*n) - q^(38 + 11*n) - q^(39 + 11*n) + 
     q^(41 + 12*n) + q^(42 + 12*n) + q^(43 + 12*n)), 
   (-q^(19 + 5*n) + q^(20 + 6*n) + q^(21 + 6*n) + q^(22 + 6*n) - q^(20 + 7*n) - q^(24 + 7*n) + q^(27 + 7*n) + q^(23 + 8*n) + # f(n+1)
     2*q^(24 + 8*n) + 2*q^(25 + 8*n) + 2*q^(26 + 8*n) + 2*q^(27 + 8*n) + q^(28 + 8*n) - q^(30 + 8*n) - q^(31 + 8*n) - q^(27 + 9*n) - 
     q^(28 + 9*n) - q^(29 + 9*n) + q^(32 + 9*n) + q^(33 + 9*n) + q^(34 + 9*n) + q^(30 + 10*n) + q^(31 + 10*n) + q^(32 + 10*n) + 
     q^(33 + 10*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(35 + 11*n) + q^(36 + 11*n) + q^(37 + 11*n) + q^(38 + 11*n) + q^(39 + 11*n) + 
     q^(40 + 11*n) - q^(38 + 12*n) - q^(39 + 12*n) - q^(40 + 12*n) + q^(42 + 13*n) + q^(43 + 13*n) + q^(44 + 13*n) + q^(45 + 13*n) + 
     q^(46 + 13*n) + q^(47 + 13*n)), 
   (q^(12 + 3*n) + q^(13 + 3*n) - 2*q^(15 + 4*n) - 3*q^(16 + 4*n) - 2*q^(17 + 4*n) - q^(18 + 4*n) + q^(18 + 5*n) + 2*q^(19 + 5*n) + # f(n+2)
     2*q^(20 + 5*n) + q^(21 + 5*n) + q^(22 + 5*n) - q^(19 + 6*n) - q^(20 + 6*n) - 3*q^(21 + 6*n) - 3*q^(22 + 6*n) - 3*q^(23 + 6*n) - 
     q^(24 + 6*n) - q^(25 + 6*n) + q^(21 + 7*n) + 2*q^(22 + 7*n) + 3*q^(23 + 7*n) + 3*q^(24 + 7*n) + 3*q^(25 + 7*n) + q^(26 + 7*n) - 
     2*q^(28 + 7*n) - q^(29 + 7*n) - q^(30 + 7*n) - q^(24 + 8*n) - 2*q^(25 + 8*n) - 3*q^(26 + 8*n) - 2*q^(27 + 8*n) - 3*q^(28 + 8*n) - 
     q^(29 + 8*n) + 2*q^(31 + 8*n) + 2*q^(32 + 8*n) + q^(33 + 8*n) + q^(27 + 9*n) + 3*q^(28 + 9*n) + 4*q^(29 + 9*n) + 4*q^(30 + 9*n) + 
     2*q^(31 + 9*n) + q^(32 + 9*n) - 2*q^(33 + 9*n) - 2*q^(34 + 9*n) - 4*q^(35 + 9*n) - 2*q^(36 + 9*n) - q^(37 + 9*n) - q^(31 + 10*n) - 
     q^(32 + 10*n) - q^(33 + 10*n) + q^(34 + 10*n) + 2*q^(35 + 10*n) + 3*q^(36 + 10*n) + 3*q^(37 + 10*n) + 2*q^(38 + 10*n) + q^(39 + 10*n) + 
     q^(35 + 11*n) - 2*q^(38 + 11*n) - 2*q^(39 + 11*n) - 3*q^(40 + 11*n) - 2*q^(41 + 11*n) - q^(42 + 11*n) + q^(39 + 12*n) + 2*q^(40 + 12*n) + 
     3*q^(41 + 12*n) + 3*q^(42 + 12*n) + 2*q^(43 + 12*n) + q^(44 + 12*n) - q^(43 + 13*n) - q^(44 + 13*n) - q^(45 + 13*n) + q^(47 + 14*n) + 
     q^(48 + 14*n) + q^(49 + 14*n)), 
   (q^(10 + 2*n) + q^(12 + 3*n) - q^(12 + 4*n) - 2*q^(13 + 4*n) - 4*q^(14 + 4*n) - 5*q^(15 + 4*n) - 5*q^(16 + 4*n) - 2*q^(17 + 4*n) - # f(n+3)
     q^(18 + 4*n) + q^(15 + 5*n) + 2*q^(16 + 5*n) + 4*q^(17 + 5*n) + 5*q^(18 + 5*n) + 6*q^(19 + 5*n) + 5*q^(20 + 5*n) + 3*q^(21 + 5*n) + 
     q^(22 + 5*n) - q^(18 + 6*n) - 4*q^(19 + 6*n) - 6*q^(20 + 6*n) - 8*q^(21 + 6*n) - 7*q^(22 + 6*n) - 7*q^(23 + 6*n) - 4*q^(24 + 6*n) - 
     2*q^(25 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + 2*q^(24 + 7*n) + q^(25 + 7*n) - q^(28 + 7*n) + q^(25 + 8*n) + q^(26 + 8*n) + 
     2*q^(27 + 8*n) + 2*q^(28 + 8*n) + 3*q^(29 + 8*n) + 3*q^(30 + 8*n) + 3*q^(31 + 8*n) + 2*q^(32 + 8*n) - q^(28 + 9*n) - 3*q^(29 + 9*n) - 
     5*q^(30 + 9*n) - 7*q^(31 + 9*n) - 7*q^(32 + 9*n) - 7*q^(33 + 9*n) - 5*q^(34 + 9*n) - 4*q^(35 + 9*n) - q^(36 + 9*n) + q^(32 + 10*n) + 
     3*q^(33 + 10*n) + 5*q^(34 + 10*n) + 6*q^(35 + 10*n) + 4*q^(36 + 10*n) + 3*q^(37 + 10*n) + q^(38 + 10*n) + q^(39 + 10*n) - q^(36 + 11*n) - 
     2*q^(37 + 11*n) - 3*q^(38 + 11*n) - 2*q^(39 + 11*n) - q^(40 + 11*n) - q^(41 + 12*n) - q^(42 + 12*n) - 2*q^(43 + 12*n) - q^(44 + 12*n) - 
     q^(45 + 12*n) + q^(44 + 13*n) + 2*q^(45 + 13*n) + 2*q^(46 + 13*n) + q^(47 + 13*n) - q^(48 + 14*n) - q^(49 + 14*n) - q^(50 + 14*n)),
    (-1 + q^(2 + n) + 2*q^(3 + n) + 2*q^(4 + n) - q^(5 + 2*n) - 2*q^(6 + 2*n) - 3*q^(7 + 2*n) - q^(8 + 2*n) + q^(7 + 3*n) + q^(8 + 3*n) + # f(n+4)
     2*q^(9 + 3*n) + q^(10 + 3*n) + q^(11 + 3*n) - q^(13 + 5*n) - q^(14 + 5*n) - 2*q^(15 + 5*n) - q^(16 + 5*n) - q^(17 + 5*n) + q^(16 + 6*n) + 
     2*q^(17 + 6*n) + 2*q^(18 + 6*n) + q^(19 + 6*n) - q^(20 + 7*n) - q^(21 + 7*n) - q^(22 + 7*n))
]

In [17]:
def coeff_to_operator(coeff, dest_q, dest_Q):
    import re
    opers = coeff.factor().operands()
    if len(opers) == 2:
        big, small = opers
        extra = 1
    else:
        big, small, extra = opers
    string_big = str(big.simplify_full())
    string_small = str(small.simplify_full())
    regs = [(r"q\^\((\d*)\*n\)", r"Q^(\1)"),
            (r"q\^\((\d*)\*n \+ (\d*)\)", r"q^(\2)*Q^(\1)"),
            (r"q\^\(n \+ (\d*)\)", r"q^(\1)*Q^(1)"),
            (r"q\^n", r"Q^(1)"),
            (r"\^", r"**")]
    def apply_regs(regs, string):
        for reg in regs:
            string = re.sub(*reg, string)
        return string
    
    q = dest_q; Q = dest_Q
    return dest_Q.parent()(extra)*eval(apply_regs(regs, string_big)) * eval(apply_regs(regs, string_small))

In [15]:
coeffs = [
    -q^(3*n)*(1 + q^n)*(-q + q^n)*(-q^3 + q^(2*n))*(1 - q^n + q^(2*n)), # f(n-2)
    -q^(2 + 2*n)*(q^2 + q^(3*n) - q^(5*n) + q^(6*n) + q^(2 + n) + q^(3 + n) - q^(1 + 2*n) - q^(2 + 2*n) + q^(1 + 3*n) + q^(1 + 4*n) +  q^(2 + 4*n) - q^(1 + 5*n)), # f(n-1)
    q^2*(q^2 + q^(2*n) - q^(1 + n)) #f(n)
]

In [12]:
L

q^4 - q^3*Q + (-q^4+q^2)*Q^2 + (-q^5-q^4)*Q^3 + (q^4+q^3)*Q^4 + (-q^3-q^2)*Q^5 - q^5*Q^3*Si^2 + (-q^4-q^3)*Q^6 + q^5*Q^4*Si^2 + (q^3+q^2)*Q^7 + (-q^5+q^3+q^2)*Q^5*Si^2 - q^2*Q^8 + (-q^3-q^2)*Q^6*Si^2 + (q^3+q^2-1)*Q^7*Si^2 + Q^8*Si^2 - Q^9*Si^2

Second (and smaller example):

In [37]:
coeffs2 = [
    q^(3*n + 6)*(1 - q^(1 + n))*(q + q^(3*n + 6) - q^(3 + n)),
    q^(-3 + n)*(-q^7 + q^(8*n + 16) + q^(8 + n) + q^(9 + n) + q^(8 + 2*n) +
        q^(9 + 2*n) - q^(9 + 3*n) - 2*q^(10 + 3*n) - 2*q^(11 + 3*n) - 
        q^(12 + 3*n) + q^(11 + 4*n) + q^(12 + 4*n) + q^(13 + 4*n) + 
        q^(11 + 5*n) + q^(12 + 5*n) + q^(13 + 5*n) + q^(14 + 5*n) - 
        q^(13 + 6*n) - q^(14 + 6*n) - q^(15 + 6*n)),
    -(q^4 + q^(3*(n+2)) - q^(5 + n))    
]

In [38]:
LL = sum([coeff_to_operator(coeffs2[i], A(q), A(Q))*S^i for i in range(len(coeffs2))]);

ValueError: too many values to unpack (expected 3)

## Generic $q$-binomial coefficients

In [21]:
B2 = QBinomialBasis(a=2)

In [22]:
B2.mult_in(B2.QPower()).functional_matrix(5)

[                                                                                                                                       1                                                                                                                                        q                                                                                                                                      q^2                                                                                                                                      q^3                                                                                                                                      q^4]
[                                                                                                                                       0                                                                                                                                  q^2 + q                                                                                                                    q^5 + q^4 + q^3 + q^2                                                                                                        q^8 + q^7 + q^6 + q^5 + q^4 + q^3                                                                                          q^11 + q^10 + q^9 + q^8 + q^7 + q^6 + q^5 + q^4]
[                                                                                                                                       0                                                                                                                                        q                                                                                                            q^6 + q^5 + 2*q^4 + q^3 + q^2                                                                          q^11 + q^10 + 2*q^9 + 2*q^8 + 3*q^7 + 2*q^6 + 2*q^5 + q^4 + q^3                                     q^16 + q^15 + 2*q^14 + 2*q^13 + 3*q^12 + 3*q^11 + 4*q^10 + 3*q^9 + 3*q^8 + 2*q^7 + 2*q^6 + q^5 + q^4]
[                                                                                                                                       0                                                                                                                                        0                                                                                                                    q^5 + q^4 + q^3 + q^2                                                                 q^12 + q^11 + 2*q^10 + 3*q^9 + 3*q^8 + 3*q^7 + 3*q^6 + 2*q^5 + q^4 + q^3          q^19 + q^18 + 2*q^17 + 3*q^16 + 4*q^15 + 5*q^14 + 6*q^13 + 6*q^12 + 6*q^11 + 6*q^10 + 5*q^9 + 4*q^8 + 3*q^7 + 2*q^6 + q^5 + q^4]
[                                                                                                                                       0                                                                                                                                        0                                                                                                                                      q^2                                                                          q^11 + q^10 + 2*q^9 + 2*q^8 + 3*q^7 + 2*q^6 + 2*q^5 + q^4 + q^3 q^20 + q^19 + 2*q^18 + 3*q^17 + 5*q^16 + 5*q^15 + 7*q^14 + 7*q^13 + 8*q^12 + 7*q^11 + 7*q^10 + 5*q^9 + 5*q^8 + 3*q^7 + 2*q^6 + q^5 + q^4]

In [27]:
Matrix([[((B2.QPower(2) + B2.QPower())*B2.shift(1))[i][j] for j in range(5)] for i in range(5)])

[                                                                                                                                                                         0                                                                                                                                                                    2*q + 2                                                                                                                                                    2*q^3 + 2*q^2 + 2*q + 2                                                                                                                                    2*q^5 + 2*q^4 + 2*q^3 + 2*q^2 + 2*q + 2                                                                                                                    2*q^7 + 2*q^6 + 2*q^5 + 2*q^4 + 2*q^3 + 2*q^2 + 2*q + 2]
[                                                                                                                                                                         0                                                                                                                                                                    q^2 + q                                                                                                                                    q^6 + 2*q^5 + 3*q^4 + 3*q^3 + 2*q^2 + q                                                                                                   q^10 + 2*q^9 + 3*q^8 + 4*q^7 + 5*q^6 + 5*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q                                                               q^14 + 2*q^13 + 3*q^12 + 4*q^11 + 5*q^10 + 6*q^9 + 7*q^8 + 7*q^7 + 6*q^6 + 5*q^5 + 4*q^4 + 3*q^3 + 2*q^2 + q]
[                                                                                                                                                                         0                                                                                                                                                                          0                                                                                                                                      q^7 + q^6 + 2*q^5 + 2*q^4 + q^3 + q^2                                                                                  q^13 + q^12 + 3*q^11 + 4*q^10 + 5*q^9 + 6*q^8 + 6*q^7 + 5*q^6 + 4*q^5 + 3*q^4 + q^3 + q^2                      q^19 + q^18 + 3*q^17 + 4*q^16 + 6*q^15 + 8*q^14 + 10*q^13 + 11*q^12 + 12*q^11 + 12*q^10 + 11*q^9 + 10*q^8 + 8*q^7 + 6*q^6 + 4*q^5 + 3*q^4 + q^3 + q^2]
[                                                                                                                                                                         0                                                                                                                                                                          0                                                                                                                                                                  q^6 + q^3                                                                                 q^14 + q^13 + 2*q^12 + 3*q^11 + 4*q^10 + 4*q^9 + 4*q^8 + 4*q^7 + 3*q^6 + 2*q^5 + q^4 + q^3 q^22 + q^21 + 2*q^20 + 4*q^19 + 6*q^18 + 7*q^17 + 10*q^16 + 12*q^15 + 13*q^14 + 14*q^13 + 14*q^12 + 13*q^11 + 12*q^10 + 10*q^9 + 7*q^8 + 6*q^7 + 4*q^6 + 2*q^5 + q^4 + q^3]
[                                                                                                                                                                         0                                                                                                                                                                          0                                                                                                                                                                          0                                                           